# Investigating the Data

In [ ]:
%load_ext autoreload
%autoreload 2

from plot_fuse_results import (plot_results, plot_results2, get_merged_table, get_bohlin78)
from matplotlib import pyplot

pyplot.ioff()

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 250
#%matplotlib widget

# Get the data

### data for Gordon09 extinction curve sightlines

In [ ]:
data = get_merged_table()

### data for Gordon09 comparision star sightlines

In [ ]:
data_comp = get_merged_table(comp=True)

### data for Bohlin 78 Copernicus sightlines

In [ ]:
data_bohlin78 = get_bohlin78()

# Gas-to-A(V)

In [ ]:
plot_results2(data, 'AV', 'nhtot', pxrange=[0.0,3.5], pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

# Gas-to-E(B-V)

In [ ]:
plot_results2(data, 'EBV', 'nhtot', pyrange=[0.0,0.8e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

## Gas-to-A(V) versus dust column

In [ ]:
plot_results2(data, 'AV', 'NH_AV', pyrange=[0.0,0.5e22], pxrange=[0,4],
            data_comp=data_comp,data_bohlin=data_bohlin78)

## Gas-to-E(B-V) versus dust column

In [ ]:
plot_results2(data, 'EBV', 'NH_EBV', pyrange=[0.0,1.5e22], pxrange=[-.1,1],
            data_comp=data_comp,data_bohlin=data_bohlin78)

# Gas-to-A(V) versus average grain size

In [ ]:
plot_results2(data, 'RV', 'NH_AV', pxrange=[2.5,6.0], pyrange=[0.0,0.5e22],
            data_comp=data_comp)

# Gas-to-A(V) versus molecular fraction

In [ ]:
plot_results2(data, 'fh2', 'NH_AV', pxrange=[0.0,0.7], pyrange=[0.0,0.5e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

# Gas-to-E(B-V) versus molecular fraction

In [ ]:
plot_results2(data, 'fh2', 'NH_EBV', pxrange=[0.0,0.7], pyrange=[0.0,1.5e22],
            data_comp=data_comp, data_bohlin=data_bohlin78)

# molecular fraction versus E(B-V) dust column

In [ ]:
plot_results2(data, 'EBV', 'fh2',
            data_comp=data_comp, data_bohlin=data_bohlin78)

# molecular fraction versus A(V) dust column

In [ ]:
plot_results2(data, 'AV', 'fh2',
            data_comp=data_comp, data_bohlin=data_bohlin78)

# molecular fraction versus average grain size

In [ ]:
plot_results2(data, 'RV', 'fh2', pxrange=[2.0,6.0],
            data_comp=data_comp)

longitude versus rv and distance versus rv (and versus fh2)

COS FUV needs 2x10-13 ergs/blah blah

E140H for CO (maybe E140M), S/N 15...

E(B-V)/distance versus fH2.  Should also do A(V)/d.  Maybe n(H) as well.

In [ ]:
plot_results2(data, 'EBV', 'RV')

In [ ]:
plot_results2(data, 'AV', 'RV')